In [8]:
#Check if dataset loads
import csv
import os
import numpy as np
import cv2
import keras

DATASET_PATH = "../../data/udacity_sim_data/"
VAL_PATH = "../../data/track2/seq1/"
def load_dataset(file_path):
    '''
    Loads dataset in memory
    '''
    dataset = []
    with open(file_path) as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            try:
                dataset.append({'center':line[0], 'left':line[1], 'right':line[2], 'steering':float(line[3]), 
                            'throttle':float(line[4]), 'brake':float(line[5]), 'speed':float(line[6])/15.0 - 1})
            except:
                continue
    return dataset

dataset = load_dataset(os.path.join(DATASET_PATH, "driving_log.csv"))
val_dataset = load_dataset(os.path.join(VAL_PATH, "driving_log.csv"))

print("Loaded {} samples from file {}".format(len(dataset),DATASET_PATH))
print("Loaded {} samples from file {}".format(len(val_dataset),VAL_PATH))

Loaded 45999 samples from file ../../data/udacity_sim_data/
Loaded 1328 samples from file ../../data/track2/seq1/


In [9]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

sample_file = os.path.join(DATASET_PATH, dataset[0]['center'].strip())
INPUT_IMAGE_ROWS, INPUT_IMAGE_COLS, INPUT_IMAGE_CHANNELS = 160, 320, 1

seq_len = 10
BATCH_SIZE = 10

train_set = dataset
valid_set = val_dataset

print("train set has {} elements".format(len(train_set)))
print("valid set has {} elements".format(len(valid_set)))

def data_generator(path, dataset, seq_len):
    batch_seq_images = np.zeros((BATCH_SIZE, seq_len, 160, 320, 1))
    batch_seq_speed = np.zeros((BATCH_SIZE, seq_len, 1))
    
    while 1:
        for i in range(BATCH_SIZE):
            while 1:
                index = np.random.randint(len(dataset))
                if index + seq_len <= len(dataset):
                    seq_speed = []
                    seq_images = []
                    for j in range(index, index + seq_len):
                        seq_speed.append(dataset[j]["speed"])
                        img = cv2.imread(path + dataset[j]["center"].strip())
                        img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)[:, :, 1]
                        img = np.asarray(img).reshape(160, 320, 1)
                        seq_images.append(img)
                    seq_speed = np.array(seq_speed)
                    seq_images = np.array(seq_images)
                    break
                else:
                    continue
            batch_seq_images[i] = seq_images
            batch_seq_speed[i] = seq_speed.reshape(seq_len, 1)
            #batch_seq_steering_angles[i] = seq_steering_angles

        # for ru
        yield batch_seq_images, batch_seq_speed[:, -1, :]


train set has 45999 elements
valid set has 1328 elements


In [10]:
sample_gen = data_generator(DATASET_PATH, train_set, seq_len)
x, y = next(sample_gen)
print(y)

[[ 0.74089933]
 [ 0.24607   ]
 [ 1.002306  ]
 [ 0.26081867]
 [ 0.57009267]
 [ 1.00216133]
 [ 1.00148733]
 [-0.38371633]
 [ 0.45018867]
 [ 0.11120867]]


In [11]:
from keras.models import Sequential, Model
from keras.layers.core import Lambda, Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Cropping2D, Convolution2D, MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers.recurrent import GRU, LSTM
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam, RMSprop

print("\nBuilding and compiling the model ...")

model = Sequential()
# Preprocess incoming data, centered around zero with small standard deviation 
model.add(TimeDistributed(Lambda(lambda x: (x / 127.5) - 1.0), input_shape=(None, INPUT_IMAGE_ROWS, INPUT_IMAGE_COLS, INPUT_IMAGE_CHANNELS)))
# Block - conv
model.add(TimeDistributed(Convolution2D(1, 11, 11, border_mode='valid', subsample=[2, 2], init='glorot_uniform', activation='relu', name='Conv1')))
# Block - conv
model.add(TimeDistributed(Convolution2D(1, 5, 5, border_mode='valid', subsample=[2, 2], init='glorot_uniform', activation='relu', name='Conv2')))
# Block - conv
#model.add(Convolution2D(1, 3, 3, border_mode='valid', subsample=[2, 2], init='glorot_uniform', activation='relu', name='Conv3'))
# Block - conv
#model.add(Convolution2D(1, 3, 3, border_mode='valid', init='glorot_uniform', activation='relu', name='Conv4'))
# Block - flatten
#model.add(TimeDistributed(MaxPooling2D((4,4),(4,4),'valid', name='pool1')))
model.add(TimeDistributed(Flatten(name='flat')))
model.add(Activation('relu'))
#model.add(TimeDistributed(Dropout(0.20)))
model.add(GRU(16, return_sequences=False, init='glorot_uniform', inner_init='glorot_uniform', activation='relu',
             name='GRU1'))
model.add(Dropout(0.20))
model.add(Dense(1, name='output')) 
model.summary()

#adam = Adam(lr=0.001)
rmsprop = RMSprop()
model.compile(loss='mean_squared_error', optimizer=rmsprop)


Building and compiling the model ...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
timedistributed_11 (TimeDistribu (None, None, 160, 320 0           timedistributed_input_3[0][0]    
____________________________________________________________________________________________________
timedistributed_12 (TimeDistribu (None, None, 75, 155, 122         timedistributed_11[0][0]         
____________________________________________________________________________________________________
timedistributed_13 (TimeDistribu (None, None, 36, 76,  26          timedistributed_12[0][0]         
____________________________________________________________________________________________________
timedistributed_14 (TimeDistribu (None, None, 2736)    0           timedistributed_13[0][0]         
_____________________________________________________

In [12]:
import keras
from keras.callbacks import Callback
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
import math

class LifeCycleCallBack(keras.callbacks.Callback):
    
    def on_epoch_begin(self, epoch, logs={}):
        pass
    
    def on_epoch_end(self, epoch, logs={}):
        pass

    def on_batch_begin(self, batch, logs={}):
        pass
    
    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        
    def on_train_begin(self, logs={}):
        print('Beginning training')
        self.losses = []
        
    def on_train_end(self, logs={}):
        print('Ending Training')


NUM_EPOCHS = 250
        
lifecycle_callback = LifeCycleCallBack()

tensorboard = keras.callbacks.TensorBoard(log_dir='../../data/logs/speed_cnn_rnn/', histogram_freq=1, 
                                          write_graph=True, write_images=True)

checkpoint = keras.callbacks.ModelCheckpoint("../../data/weights/speed_cnn_rnn/model.h5", monitor='val_loss', verbose=0, 
                                             save_best_only=True, save_weights_only=True, mode='auto', period=1)

train_generator = data_generator(DATASET_PATH, train_set, seq_len)
valid_generator = data_generator(VAL_PATH, valid_set, seq_len)

samples_per_epoch = math.ceil((len(train_set) - seq_len)/(seq_len*BATCH_SIZE))*BATCH_SIZE
nb_val_samples = math.ceil((len(valid_set) - seq_len)/(seq_len*BATCH_SIZE))*BATCH_SIZE

print("\nSaving Model...")

model_json = model.to_json()
with open("../../data/weights/speed_cnn_rnn/model.json", "w") as json_file:
    json_file.write(model_json)

#model.save_weights("model.h5")
print("Model Saved.")


history = model.fit_generator(train_generator,
                              validation_data=valid_generator,
                              samples_per_epoch=samples_per_epoch,
                              nb_val_samples=nb_val_samples,
                              nb_epoch=NUM_EPOCHS, verbose=1, 
                              callbacks=[lifecycle_callback, checkpoint])

print("\nTraining the model ended.")



Saving Model...
Model Saved.
Beginning training
Epoch 1/250
4600/4600 [==============================] - 54s - loss: 0.2206 - val_loss: 0.1582
Epoch 2/250
4600/4600 [==============================] - 53s - loss: 0.1622 - val_loss: 0.2096
Epoch 3/250
4600/4600 [==============================] - 53s - loss: 0.1454 - val_loss: 0.2170
Epoch 4/250
4600/4600 [==============================] - 53s - loss: 0.1300 - val_loss: 0.1604
Epoch 5/250
4600/4600 [==============================] - 53s - loss: 0.1247 - val_loss: 0.1681
Epoch 6/250
4600/4600 [==============================] - 53s - loss: 0.1087 - val_loss: 0.1672
Epoch 7/250
4600/4600 [==============================] - 53s - loss: 0.0975 - val_loss: 0.1212
Epoch 8/250
4600/4600 [==============================] - 53s - loss: 0.0872 - val_loss: 0.1895
Epoch 9/250
4600/4600 [==============================] - 53s - loss: 0.0836 - val_loss: 0.1657
Epoch 10/250
4600/4600 [==============================] - 53s - loss: 0.0770 - val_loss: 0.1632


KeyboardInterrupt: 